***IMPORT DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE***

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')

import os 
from datetime import datetime
import pandas as pd
import Connexion_Transfert as ct

# DONNEES BRUIT

## Lecture des fichiers ods

In [3]:
# test de formatage sur un fichier
dossier = r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono1_10RuePierreRonsard'
nomFichier = 'sono1_01-04-2022.csv'
fichier = os.path.join(dossier, nomFichier)

In [4]:
# options de formatage
def conversionNumerique(numTexte):
    return float(numTexte.replace(',','.'))


def conversionDate(dateTexte):
    return datetime.strptime(dateTexte, "%H:%M:%S:%f %d/%m/%Y")


colonnes = ['date_heure', 'leq_a', 'leq_lin', 'crete_c', 'fast_inst_a', 'fast_max_a', 'fast_min_a', 6, 8, 10, 12, 16,
            20, 25, 31, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400, 500,
            630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000, 10000,
            12500, 16000, 20000]
converters = {'leq_a': conversionNumerique, 'leq_lin': conversionNumerique, 'crete_c': conversionNumerique, 'fast_inst_a': conversionNumerique, 
              'fast_max_a': conversionNumerique, 'fast_min_a': conversionNumerique, 'to_6.3': conversionNumerique, 'to_8': conversionNumerique,
              'to_10': conversionNumerique, 'to_12': conversionNumerique, 'to_16': conversionNumerique, 'to_20': conversionNumerique, 
              'to_25': conversionNumerique, 'to_31': conversionNumerique, 'to_40': conversionNumerique, 'to_50': conversionNumerique, 'to_63': conversionNumerique,
              'to_80': conversionNumerique, 'to_100': conversionNumerique, 'to_125': conversionNumerique, 'to_160': conversionNumerique, 'to_200': conversionNumerique,
              'to_250': conversionNumerique, 'to_315': conversionNumerique, 'to_400': conversionNumerique, 'to_500': conversionNumerique,'to_630': conversionNumerique,
              'to_800': conversionNumerique, 'to_1000': conversionNumerique, 'to_1250': conversionNumerique, 'to_1600': conversionNumerique,
              'to_2000': conversionNumerique, 'to_2500': conversionNumerique, 'to_3150': conversionNumerique,'to_4000': conversionNumerique,
              'to_5000': conversionNumerique, 'to_6300': conversionNumerique, 'to_8000': conversionNumerique, 'to_10000': conversionNumerique,
              'to_12500': conversionNumerique, 'to_16000': conversionNumerique, 'to_20000': conversionNumerique}
# lecture
dfBruteFichier = pd.read_csv(fichier, skiprows=49, names=colonnes, parse_dates=[0], dayfirst=True, date_parser=conversionDate, converters=converters)

In [8]:
# mise en forme
dfSpectreBrute = dfBruteFichier[[c for c in dfBruteFichier.columns if isinstance(c, int)]].stack().reset_index()
dfSpectreBrute.columns = ['id', 'freq', 'valeur']
dfSpectreBrute['type_freq'] = 'tiers octave'

In [10]:
dfSpectreBrute.head(5)

,id,freq,valeur,type_freq
0,0,6,"37,6",tiers octave
1,0,8,45,tiers octave
2,0,10,"28,4",tiers octave
3,0,12,"7,7",tiers octave
4,0,16,"58,2",tiers octave


In [12]:
dfNiveauBruit = dfBruteFichier[[c for c in dfBruteFichier.columns if not isinstance(c, int)]]

In [13]:
dfNiveauBruit

,date_heure,leq_a,leq_lin,crete_c,fast_inst_a,fast_max_a,fast_min_a
0,2022-04-01 00:00:33.000,48.3,55.0,64.0,48.0,48.0,48.0
1,2022-04-01 00:00:33.125,47.3,56.7,64.6,47.5,48.1,47.5
2,2022-04-01 00:00:33.250,47.3,61.8,65.2,47.3,47.7,47.3
3,2022-04-01 00:00:33.375,46.8,60.3,64.2,47.0,47.3,47.0
4,2022-04-01 00:00:33.500,46.4,59.7,63.7,46.7,47.0,46.6
...,...,...,...,...,...,...,...
690931,2022-04-01 23:59:59.375,51.6,64.0,70.4,51.4,51.5,51.2
690932,2022-04-01 23:59:59.500,51.0,62.3,69.4,51.2,51.4,51.1
690933,2022-04-01 23:59:59.625,51.0,59.6,67.6,51.1,51.3,50.9
690934,2022-04-01 23:59:59.750,51.6,60.8,69.2,51.3,51.5,51.0


In [26]:
with ct.ConnexionBdd('ech24') as c:
    indexMaxBruit = pd.read_sql("SELECT CASE WHEN max(id) IS NULL THEN 0 ELSE max(id) END +1 AS max_id FROM mesures_physiques.niveau_bruit", 
                                c.sqlAlchemyConn).iloc[0, 0]